In [1]:
!unzip /content/videosfolder.zip

Archive:  /content/videosfolder.zip
   creating: videosfolder/
  inflating: videosfolder/lat pulldown_1.mp4  
  inflating: videosfolder/lat pulldown_2.mp4  
  inflating: videosfolder/lat pulldown_39.mp4  
  inflating: videosfolder/push-up_39.mp4  
  inflating: videosfolder/push-up_46.mp4  
  inflating: videosfolder/romanian deadlift_6.mp4  


In [2]:
! git clone https://github.com/v-iashin/video_features.git

Cloning into 'video_features'...
remote: Enumerating objects: 1462, done.
remote: Counting objects: 100% (406/406), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 1462 (delta 313), reused 284 (delta 249), pack-reused 1056 (from 2)
Receiving objects: 100% (1462/1462), 288.84 MiB | 29.73 MiB/s, done.
Resolving deltas: 100% (800/800), done.
Updating files: 100% (100/100), done.


In [3]:
! pip install --upgrade pip==23.3.1
! pip install omegaconf==2.0.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
%cd video_features

/content/video_features


In [5]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

'Tesla T4'

In [7]:
import numpy as np
import os
import json


def extract_features_from_folder(video_folder, save_path, feature_type='i3d', flow_type='raft'):
    os.makedirs(save_path, exist_ok=True)

    args = OmegaConf.load(build_cfg_path(feature_type))
    args.flow_type = flow_type

    extractor = ExtractI3D(args)

    all_annotations = {"database": {}}

    video_files = [f for f in os.listdir(video_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

    for i, video_file in enumerate(video_files, 1):
        video_path = os.path.join(video_folder, video_file)

        print(f"Processing video {i} of {len(video_files)}: {video_file}")

        args.video_paths = [video_path]
        feature_dict = extractor.extract(video_path)

        video_name = os.path.splitext(video_file)[0]

        video_save_path = os.path.join(save_path, video_name)
        os.makedirs(video_save_path, exist_ok=True)

        rgb_features = None
        flow_features = None

        timestamps_ms = None

        for key, value in feature_dict.items():
            if key == 'timestamps_ms':
                timestamps_ms = value

            if key in ["rgb", "flow"]:
                output_filename = f"{key}_features.npy"
                full_output_path = os.path.join(video_save_path, output_filename)
                np.save(full_output_path, value)

                if key == "rgb":
                    rgb_features = value
                elif key == "flow":
                    flow_features = value

        if rgb_features is not None and flow_features is not None:
            combined_features = np.hstack((flow_features, rgb_features))

            if combined_features.size == 0:
                continue

            combined_path = os.path.join('/content/extract_videos', f"{video_name}.npy")
            np.save(combined_path, combined_features)

            num_segments = feature_dict.get("rgb", feature_dict.get("flow", None)).shape[0]

            action = video_name.split('_')[0]

            fps = 30.0
            if timestamps_ms is not None and len(timestamps_ms) > 0:
                last_timestamp_ms = timestamps_ms[-1]
                duration = last_timestamp_ms / 1000
            else:
                duration = 0

            annotation = create_annotation_json(
                video_file,
                duration,
                fps,
                [{"label": action,
                  "segment": [0.0, duration],
                  "segment(frames)": [0, num_segments],
                  "label_id": get_label_id(action)}]
            )

            all_annotations["database"][video_name] = annotation

    annotation_path = os.path.join('/content/extract_videos', 'annotations.json')
    with open(annotation_path, 'w') as f:
        json.dump(all_annotations, f, indent=4)

    return all_annotations


def create_annotation_json(video_file, duration, fps, action_segments):
    annotation = {
        "subset": "training",
        "duration": float(duration),
        "fps": float(fps),
        "annotations": action_segments
    }
    return annotation


def get_label_id(action_name):
    action_to_id = {
        "cricket": 1,
        "basketball": 2,
        "football": 3,
        "barbell biceps curl": 100,
        "bench press" : 101,
        "chest fly machine": 102,
        "deadlift": 103,
        "decline bench press": 104,
        "hammer curl": 105,
        "hip thrust": 106,
        "incline bench press": 107,
        "lat pulldown": 108,
        "lateral raise": 109,
        "leg extension": 110,
        "leg raises": 111,
        "pull Up": 112,
        "push-up": 113,
        "romanian deadlift": 114,
        "russian twist": 115,
        "shoulder press": 116,
        "squat": 117,
        "t bar row": 118,
        "tricep dips": 119,
        "tricep Pushdown": 120,
    }

    return action_to_id.get(action_name.lower(), 0)


if __name__ == "__main__":
    video_folder = '/content/videosfolder'
    save_path = '/content/features_and_annotations'

    annotations = extract_features_from_folder(video_folder, save_path)

    print("Feature extraction and annotation complete!")


Processing video 1 of 6: push-up_46.mp4
Processing video 2 of 6: lat pulldown_2.mp4
Processing video 3 of 6: lat pulldown_1.mp4
Processing video 4 of 6: push-up_39.mp4
Processing video 5 of 6: lat pulldown_39.mp4
Processing video 6 of 6: romanian deadlift_6.mp4
Feature extraction and annotation complete!


In [8]:
import shutil

folder_path = '/content/extract_videos'
zip_file_name = '/content/extract videos.zip'


shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', folder_path)

'/content/extract videos.zip'